<a href="https://colab.research.google.com/github/dindorios/dindorios/blob/main/BasedeDadoseTreinamentodaRedeYOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install fiftyone

In [ ]:
#Treinar o modelo YOLOv8
!yolo train model=yolov8n.pt data=coco.yaml epochs=50 imgsz=640 project=/content/drive/MyDrive/yolov8_weights
# Avaliar o modelo
!yolo val model=/content/drive/MyDrive/yolov8_weights/train/weights/best.pt data=coco.yaml

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import os
from google.colab import drive

# 1. Montar o Google Drive (com tratamento de erros)
try:
    drive.mount('/content/drive')
    print("Drive montado com sucesso!")
except ValueError as e:
    print(f"Erro ao montar o Drive: {e}")
    print("Verifique sua conexão e tente novamente.")
    exit() # Interrompe a execução se o Drive não for montado
except Exception as e:
    print(f"Erro inesperado ao montar o Drive: {e}")
    exit()

# 2. Definir parâmetros do dataset
dataset_name = "coco-2017"
classes_of_interest = ["person", "car", "truck", "dog", "bicycle"]
split = "train" # ou "validation", "test"
max_samples = 5000 # Ajuste conforme necessário (use um número menor para testes)
export_dir = "/content/coco_yolo_format"
yolo_format_dataset_name = f"{dataset_name}-{split}-yolo-{'-'.join(classes_of_interest)}" #Nome mais descritivo

# 3. Verificar se o dataset já existe no FiftyOne ou no formato YOLO no Drive
if fo.dataset_exists(yolo_format_dataset_name):
    print(f"Dataset YOLO formatado {yolo_format_dataset_name} já existe no FiftyOne. Carregando...")
    dataset = fo.load_dataset(yolo_format_dataset_name)
elif fo.dataset_exists(dataset_name):
  dataset = fo.load_dataset(dataset_name)
  print(f"Dataset {dataset_name} ja existe, exportando para formato YOLO...")
  dataset.export(
      export_dir=export_dir,
      dataset_type=fo.types.YOLOv5Dataset,
      classes=classes_of_interest
  )
  dataset.name = yolo_format_dataset_name
  dataset.save()
  print(f"Dataset exportado e salvo com o nome {yolo_format_dataset_name}")
else:
    print(f"Dataset {dataset_name} não encontrado. Baixando...")
    dataset = foz.load_zoo_dataset(
        dataset_name,
        split=split,
        label_types=["detections"],
        classes=classes_of_interest,
        max_samples=max_samples
    )
    dataset.name = dataset_name
    dataset.save()
    print(f"Dataset {dataset_name} carregado e salvo no FiftyOne")

    print("Exportando dataset para formato YOLO...")

    dataset.export(
      export_dir=export_dir,
      dataset_type=fo.types.YOLOv5Dataset,
      classes=classes_of_interest
    )
    dataset.name = yolo_format_dataset_name
    dataset.save()
    print(f"Dataset exportado e salvo com o nome {yolo_format_dataset_name}")

# 4. Criar pastas no Drive para os pesos (movido para antes do treinamento)
weights_path = "/content/drive/MyDrive/yolov8_weights"
if not os.path.exists(weights_path):
    os.makedirs(weights_path)
    print(f"Pasta {weights_path} criada no Drive.")
else:
    print(f"Pasta {weights_path} já existe no Drive.")

print("Processo de preparação do dataset concluído.")

In [ ]:
# Criar o arquivo YAML: Crie um arquivo coco.yaml dentro do Colab
%%writefile coco.yaml
train: /content/coco_yolo_format/train/images
val: /content/coco_yolo_format/valid/images
test: /content/coco_yolo_format/test/images

nc: 5  # Numero de classes
names: ['person', 'car', 'truck', 'dog', 'bicycle'] # Nomes das classes